In [ ]:
import os
import numpy as np
import pandas as pd
import random
import copy

from transformers import (AutoConfig, AutoModel, AutoTokenizer, AdamW, 
                          get_linear_schedule_with_warmup, logging, 
                          RobertaConfig, PreTrainedModel, RobertaModel)
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, SequentialSampler, RandomSampler, DataLoader

from tqdm.notebook import tqdm

import gc; gc.enable()
from IPython.display import clear_output

from sklearn.model_selection import StratifiedKFold

logging.set_verbosity_error()

In [ ]:
INPUT_DIR = '../input/class-level-training-data'
MODEL_NAME = 'roberta-large'

MAX_LENGTH = 256
LR = 2e-5
EPS = 1e-8

SEED = 42

NUM_FOLDS = 5
SEEDS = [113, 71, 17, 43, 37]

EPOCHS = 5
TRAIN_BATCH_SIZE = 8
VAL_BATCH_SIZE = 32

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def set_seed(seed = 0):
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    return random_state

seed=1112
random_state = set_seed(seed)

In [ ]:
class ContinuousStratifiedKFold(StratifiedKFold):
    def split(self, x, y, groups=None):
        num_bins = int(np.floor(1 + np.log2(len(y))))
        bins = pd.cut(y, bins=num_bins, labels=False)
        return super().split(x, bins, groups)

def get_data_loaders(data, fold):
    
    x_train = data.loc[data.fold != fold, 'text'].tolist()
    y_train = data.loc[data.fold != fold, 'labels'].values
    x_val = data.loc[data.fold == fold, 'text'].tolist()
    y_val = data.loc[data.fold == fold, 'labels'].values
    
    encoded_train = tokenizer.batch_encode_plus(
        x_train, 
        add_special_tokens=True, 
        return_attention_mask=True, 
        padding='max_length', 
        truncation=True,
        max_length=MAX_LENGTH, 
        return_tensors='pt'
    )
    
    encoded_val = tokenizer.batch_encode_plus(
        x_val, 
        add_special_tokens=True, 
        return_attention_mask=True, 
        padding='max_length', 
        truncation=True,
        max_length=MAX_LENGTH, 
        return_tensors='pt'
    )
    
    dataset_train = TensorDataset(
        encoded_train['input_ids'],
        encoded_train['attention_mask'],
        torch.tensor(y_train)
    )
    dataset_val = TensorDataset(
        encoded_val['input_ids'],
        encoded_val['attention_mask'],
        torch.tensor(y_val)
    )
    
    dataloader_train = DataLoader(
        dataset_train,
        sampler = RandomSampler(dataset_train),
        batch_size=TRAIN_BATCH_SIZE
    )

    dataloader_val = DataLoader(
        dataset_val,
        sampler = SequentialSampler(dataset_val),
        batch_size=VAL_BATCH_SIZE
    )

    return dataloader_train, dataloader_val

In [ ]:
class RobertaClassARegression(RobertaPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.config = config

        self.roberta = RobertaModel(config, add_pooling_layer=False)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.out_proj = nn.Linear(config.hidden_size, 1)
        self.loss = nn.MSELoss()
        
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        last_hidden_state = outputs[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        
        mean_embeddings = self.dropout(mean_embeddings)
        mean_embeddings = self.dense(mean_embeddings)
        mean_embeddings = self.dropout(mean_embeddings)
        logits = self.out_proj(mean_embeddings)

        preds = logits.squeeze(-1).squeeze(-1)
        
        if labels is not None:
            loss = torch.sqrt(self.loss(preds.view(-1).float(), labels.view(-1).float()))
            return loss
        else:
            return preds

In [ ]:
raw = pd.read_csv(os.path.join(INPUT_DIR, 'raw_data.csv'))

# G == class_a, B0 == class_b and B1 == class_c
data = copy.deepcopy(raw.loc[raw.dev_2 == 'B1'])
data.drop(data[data['labels'] == 0].index , inplace=True, axis=0)
data.reset_index(drop=True, inplace=True)
data.drop(columns=['se', 'dev_1', 'dev_2'], axis=1, inplace=True)

# Create stratified folds
kf = ContinuousStratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
for f, (t_, v_) in enumerate(kf.split(data, data.labels)):
    data.loc[v_, 'fold'] = f
data['fold'] = data['fold'].astype(int)

In [ ]:
def evaluate(model, val_dataloader):
    model.eval()
    loss_val_total = 0
    for batch in val_dataloader:
        batch = tuple(b.to(DEVICE) for b in batch)
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }
        with torch.no_grad():        
            loss = model(**inputs)
        loss_val_total += loss.item()
    loss_val_avg = loss_val_total/len(val_dataloader) 
    return loss_val_avg

def train(model, train_dataloader, val_dataloader):
    optimizer = AdamW(model.parameters(), lr = LR, eps = EPS)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * EPOCHS)
    best_val_loss = 1
    model.train()
    for epoch in range(EPOCHS):
        loss_train_total = 0
        for batch in tqdm(train_dataloader):
            model.zero_grad()
            batch = tuple(b.to(DEVICE) for b in batch)
            inputs = {
                'input_ids': batch[0],
                'attention_mask': batch[1],
                'labels': batch[2]
            }
            loss = model(**inputs)
            loss_train_total += loss.item()
            loss.backward()
            optimizer.step()
            scheduler.step()
        loss_train_avg = loss_train_total / len(train_dataloader)
        loss_val_avg = evaluate(model, val_dataloader)
        print(f'epoch:{epoch+1}/{EPOCHS} train loss={loss_train_avg}  val loss={loss_val_avg}')
        
        if loss_val_avg < best_val_loss:
            best_val_loss = loss_val_avg    
    return best_val_loss

In [ ]:
losses = []

MAX_RUNS = 2
runs = 0   # Variable to control termination condition

model = RobertaClassARegression.from_pretrained(MODEL_NAME)
model.to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

for i, seed in enumerate(SEEDS):       
    # Termination condition
    if runs == MAX_RUNS:
        print(f'{runs} runs termination condition reached.')
        break    
    
    print(f'********* seed({i}) = {seed} ***********')
    
    for fold in range(NUM_FOLDS):
        print(f'*** fold = {fold} ***')
        set_seed(seed)
        train_dataloader, val_dataloader = get_data_loaders(data, fold)
            
        loss = train(model, train_dataloader, val_dataloader)
        losses.append(loss)
        
        # Termination condition
        runs += 1
        if runs == MAX_RUNS:
            break

In [ ]:
model.save_pretrained("/kaggle/working")